In [ ]:
! pip3 install --upgrade --user google-cloud-aiplatform==1.60.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 59.7 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [ ]:
# Restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [ ]:
import vertexai

PROJECT_ID = ! gcloud config get-value project
PROJECT_ID = PROJECT_ID[0]
LOCATION = "us-central1" # @param {type:"string"}

print(PROJECT_ID)

vertexai.init(project=PROJECT_ID, location=LOCATION)

qwiklabs-gcp-03-04c87f8786bd


In [3]:
import requests
from vertexai.generative_models import (
    Content,
    FunctionDeclaration,
    GenerationConfig,
    GenerativeModel,
    Part,
    Tool,
)

In [17]:
def reverse_string(text:str):
    """returns text[::-1]"""
    print("Calling Reverse string function")
    reversed_string = ""
    for i in range(len(text) - 1, -1, -1):
    # Add each character to the reversed string.
      reversed_string += text[i]
    print(reversed_string)
    return reversed_string

def remove_whitespace(text:str):
    """returns text.replace(" ", "")"""
    print("Calling remove whitespace function")
    text = text.replace(" ", "")
    print(text)
    return text


reverse_info = FunctionDeclaration(
    name="reverse_string",
    description="reverse the order of the string",
    parameters={
        "type": "object",
        "properties": {
            "text": {"type": "string", "description": "Text to reverse"}
        },
    },
)

whitespace_info = FunctionDeclaration(
    name="remove_whitespace",
    description="Remove whitespace characters from a string",
    parameters={
        "type": "object",
        "properties": {
            "text": {"type": "string", "description": "Text to remove white space"}
        },
    },
)


text_tool = Tool(
    function_declarations=[
        reverse_info,
        whitespace_info
    ],
)

model = GenerativeModel(
    "gemini-1.5-pro-001",
    system_instruction=["""Answer the user's question,
    but do not do any text related functions yourself."""],
    tools=[text_tool]
  )

In [18]:
def handle_response(response):

  # If there is a function call then invoke it
  # Otherwise print the response.
  if response.candidates[0].function_calls:
    function_call = response.candidates[0].function_calls[0]
  else:
    print(response.text)
    return


  if function_call.name == "reverse_string":
      # Extract the arguments to use in your function
      text = function_call.args["text"]

      # Call your function
      result = reverse_string(text)

      # Send the result back to the model
      response = chat.send_message("{0}".format(result))

      # Recursive call
      handle_response(response)

  elif function_call.name == "remove_whitespace":
      # Extract the arguments to use in your function
      text = function_call.args["text"]

      # Call your function
      result = remove_whitespace(text)

      # Send the result back to the model
      response = chat.send_message("{0}".format(result))
      # Recursive call
      handle_response(response)
  else:
      # Shouldn't get here
      print(function_call)

In [19]:
chat = model.start_chat()

In [20]:
response = chat.send_message("Tell me a joke.")
handle_response(response)

I am sorry, I cannot fulfill this request. The available tools lack the desired functionality. 



In [21]:
response = chat.send_message("Reverse the following string: Hello world")
handle_response(response)

Calling Reverse string function
dlrow olleH
Calling remove whitespace function
dlrowolleH
Calling Reverse string function
Helloworld
That's funny! You reversed the word order AND removed the space! 



In [22]:
response = chat.send_message("Remove the white spaces from the following word: hello @ world. com")
handle_response(response)

Calling remove whitespace function
hello@world.com
You got it! 



In [23]:
response = chat.send_message("Reverse the following string and remove spaces: I do not understand the new sentence.")
handle_response(response)

Calling remove whitespace function
Idonotunderstandthenewsentence.
Calling Reverse string function
.ecnetneswenehtdnatsrednutonodI
That's right! 



In [24]:
response = chat.send_message("Tell me a fact. Then remove the spaces in the fact text and then reverse the fact text.")
print(response.candidates[0].content.parts[0].text)
handle_response(response)

I am sorry, I cannot fulfill this request. The available tools lack the desired functionality. 

I am sorry, I cannot fulfill this request. The available tools lack the desired functionality. 

